In [2]:
from PyLQR.sim import KDLRobot
from PyLQR.system import PosOrnPlannerSys, PosOrnKeypoint
from PyLQR.solver import BatchILQRCP, BatchILQR, ILQRRecursive
from PyLQR.utils import primitives, PythonCallbackMessage

import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

import logging
import sys

from scipy.spatial.transform import Rotation 

from cv_bridge import CvBridge, CvBridgeError

sys.path.append("/home/vdrame/catkin_ws/src/py_panda/PyPanda")
from PyPanda import Robot
from PyPanda import Utils

from utils.camera_utils import RealCamera, RealCameraROS
from utils.transform_utils import *
from utils.iLQR_wrapper import iLQR
# from utils.visualisation_utils import depth2pc
from utils.ROS_utils import generate_grasps_client, format_pointcloud_msg, run_action, get_camera_pose, gridRegistrator

import argparse
from scipy.spatial.transform import Rotation

import time

from contact_grasp.srv import contactGraspnetPointcloud2, contactGraspnetPointcloud2Response

import json
import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointField, CameraInfo, Image, PointCloud2
from std_msgs.msg import Header
from geometry_msgs.msg import PoseArray, Pose
import message_filters

import open3d as o3d

In [3]:
def restart_controller(rbt, homing=False):
    eef_pos, eef_quat = [], []
    rbt.stop_controller()
    rbt.error_recovery()
    rbt.switch_controller("joint_velocity_controller")
    if homing:
        rbt.gripper.homing()

In [4]:
bridge = CvBridge()
try:
    rospy.init_node("python_node",anonymous=True)
except:
    print("rospy already initialized")

# dispose_pos = np.array([dim, 0.66, 0.1])
# dispose_orn_wxyz = np.array([0, 1, 0.35, 0])

# Load robot
rbt = Robot("panda", use_gripper=True)
traj_gen = iLQR(rbt)

image_sub = message_filters.Subscriber("/grid_generator/disposability_grid", Image)
pose_sub = message_filters.Subscriber("/grid_generator/Posearray_pub", PoseArray)
ts = message_filters.TimeSynchronizer([pose_sub, image_sub], 1)
grid = gridRegistrator(rbt)
ts.registerCallback(grid.callback)

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth to initialise camera, otherwise image is very dark
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

difference between iLQR model ee and real robot ee
quat (zyzw) :[-2.6987786e-06 -2.2725683e-06 -2.0281452e-06  1.0000000e+00]
pos:[ 2.38809580e-06 -1.03397524e-07  3.86981814e-07]
Camera topic found


In [5]:
restart_controller(rbt)


[WARN] [1688504139.751502]: No controllers are running


In [7]:
#load pos and orn from json file
with open('config/views_pos.json') as json_file:
    views_pos = json.load(json_file)

pos_dif = 1000
keys = views_pos.keys()
angle_range = [-140, 40]
pc_fused = []
pc_colors_fused = []
reference_pose = np.eye(4)
plot_coordinate = []
for view_idx, key in enumerate(keys):
# while (not detected or not detected_with_collision):
    horizon = 30
    if abs(np.sum(np.array(rbt.model.ee_pos_rel()) - np.array(views_pos[key]["pos"]))) > 0.01:
        view_jpos, view_x_pos, view_U, view_Ks, view_ds, pos_dif, orn_dif = traj_gen.direct_trajectory(rbt.q, rbt.dq, views_pos[key]["pos"], views_pos[key]["orn_wxyz"], horizon)

        if pos_dif > 0.035:
            print("Trajectory could not reach view continue to next view")
            continue

        view_U = np.array(view_U)
        success, idx, eef_pos, eef_quat = run_action(rbt, view_U, 20)
        rbt.active_controller.send_command(np.zeros(7))
        time.sleep(0.15)
    else:
        print( np.sum(np.array(rbt.model.ee_pos_rel()) - np.array(views_pos[key]["pos"])))
        print(np.array(views_pos[key]["pos"]))
        print(key)
    
    rbg_cv, depth_cv, depth_scale = camera.get_rgb_depth()
    depth_cv = depth_cv * depth_scale
 
    current_pose = get_camera_pose(rbt, ee_depth=-0.10340-0.005)

    if view_idx == 0:
        init_pos = rbt.model.ee_pos_rel()
        init_orn_wxyz = rbt.model.ee_orn_rel()
        reference_pose = get_camera_pose(rbt, ee_depth=-0.10340-0.005)
        depth_init = depth_cv
        rgb_init = rbg_cv
        pc_fused, pc_colors_fused = depth2pc(depth_init, intrinsic, rgb_init)
        pc_fused, pc_colors_fused = regularize_pc(pc_fused, pc_colors_fused, 
                                                  downsampling_method="voxel", voxel_size=0.005,
                                                  outlier_filtering_method="radius", radius_param_arg=[25, 0.015])
    else:
        pc_fused, pc_colors_fused = add_view2pc(pc_fused, pc_colors_fused, reference_pose, current_pose, new_gbr=rbg_cv, 
                                                new_depth=depth_cv, cam_intrisic=intrinsic, regularize=True, voxel_size=0.005)

    coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    coordinate_line = draw_triangle(color=[1,0,0])

    coordinate_line.transform(current_pose)
    coordinate.transform(current_pose)
    plot_coordinate.append(coordinate)
    plot_coordinate.append(coordinate_line)

    print(plot_coordinate)
    pc_o3d = o3d.geometry.PointCloud()
    pc_o3d.points = o3d.utility.Vector3dVector(pc_fused)
    pc_o3d.colors = o3d.utility.Vector3dVector(pc_colors_fused/255)
    coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    # coordinate.translate([0,0 , 0.4])
    pc_o3d.transform(reference_pose)
    to_draw = plot_coordinate.copy()
    to_draw.append(pc_o3d)
    o3d.visualization.draw_geometries(to_draw)
    if view_idx == 2:
        break

Trajectory could not reach view continue to next view


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 3

In [14]:
current_pose

array([[ 0.36209141, -0.80969669,  0.46182365,  0.03713801],
       [-0.29578505, -0.56964332, -0.76682314,  0.59482619],
       [ 0.88396893,  0.14105952, -0.44575912,  0.16624133],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [15]:
np.asarray(pc_o3d.points)

array([[ 0.89981344, -2.09495538,  0.60591773],
       [ 0.89961087, -2.09741915,  0.60321609],
       [ 0.88341283, -2.1139854 ,  0.53865819],
       ...,
       [ 1.21074279, -1.82221269,  0.51719078],
       [ 0.0747867 ,  0.40400081, -0.01162093],
       [ 0.10828803,  0.3671978 ,  0.05573272]])